<a href="https://colab.research.google.com/github/fongbubble/fongbubble_UoB_EFIMM0142_Individual_Assessment/blob/main/Question2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [ ]:
!pip install pulp
from pulp import *
import matplotlib.pyplot as plt
import networkx as nx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 65.7 MB/s eta 0:00:00


# Question 2

## Question 2.a

In [ ]:
# Define data
locations = [
    "Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
    "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"
]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Corrected costs
cincinnati_costs = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
oakland_costs = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]

cincinnati_production_cost = 1.20
oakland_production_cost = 1.65
cincinnati_capacity = 500000

# Create LP problem
prob = pulp.LpProblem("Question2_a", LpMinimize)

# Decision variables
C = [pulp.LpVariable(f"C_{loc}", lowBound=0) for loc in locations]
O = [pulp.LpVariable(f"O_{loc}", lowBound=0) for loc in locations]

# Objective function: Minimize total cost
total_cost = (
    sum((cincinnati_costs[i] + cincinnati_production_cost) * C[i] for i in range(len(locations)))
    + sum((oakland_costs[i] + oakland_production_cost) * O[i] for i in range(len(locations)))
)
prob += total_cost

# Constraints

# Demand must be met for each location
for i in range(len(locations)):
    prob += C[i] + O[i] == demand[i], f"Demand_{locations[i]}"

# Cincinnati production capacity constraint
prob += sum(C) <= cincinnati_capacity, "Cincinnati_Capacity"

# Solve the problem
prob.solve()

# Print results
print("Status:", LpStatus[prob.status])
print(f"Total Cost: ${pulp.value(prob.objective):,.2f}")
print("\nAllocation:")
for i in range(len(locations)):
    print(
        f"{locations[i]}: Cincinnati={C[i].varValue} gallons, Oakland={O[i].varValue} gallons"
    )


Status: Optimal
Total Cost: $1,318,984.93

Allocation:
Santa Ana: Cincinnati=0.0 gallons, Oakland=22418.0 gallons
El Paso: Cincinnati=6800.0 gallons, Oakland=0.0 gallons
Pendleton: Cincinnati=39636.0 gallons, Oakland=40654.0 gallons
Houston: Cincinnati=100447.0 gallons, Oakland=0.0 gallons
Kansas City: Cincinnati=24570.0 gallons, Oakland=0.0 gallons
Los Angeles: Cincinnati=0.0 gallons, Oakland=64761.0 gallons
Glendale: Cincinnati=0.0 gallons, Oakland=33689.0 gallons
Jacksonville: Cincinnati=68486.0 gallons, Oakland=0.0 gallons
Little Rock: Cincinnati=148586.0 gallons, Oakland=0.0 gallons
Bridgeport: Cincinnati=111475.0 gallons, Oakland=0.0 gallons
Sacramento: Cincinnati=0.0 gallons, Oakland=112000.0 gallons


In [ ]:
prob

Question2_a:
MINIMIZE
1.54*C_Bridgeport + 2.04*C_El_Paso + 1001.2*C_Glendale + 1.65*C_Houston + 1.54*C_Jacksonville + 1.56*C_Kansas_City + 1.54*C_Little_Rock + 1001.2*C_Los_Angeles + 2.03*C_Pendleton + 1001.2*C_Sacramento + 1001.2*C_Santa_Ana + 1001.65*O_Bridgeport + 2.3899999999999997*O_El_Paso + 1.8699999999999999*O_Glendale + 1001.65*O_Houston + 1001.65*O_Jacksonville + 1001.65*O_Kansas_City + 1001.65*O_Little_Rock + 1.8699999999999999*O_Los_Angeles + 2.1399999999999997*O_Pendleton + 1.7999999999999998*O_Sacramento + 1.8699999999999999*O_Santa_Ana + 0.0
SUBJECT TO
Demand_Santa_Ana: C_Santa_Ana + O_Santa_Ana = 22418

Demand_El_Paso: C_El_Paso + O_El_Paso = 6800

Demand_Pendleton: C_Pendleton + O_Pendleton = 80290

Demand_Houston: C_Houston + O_Houston = 100447

Demand_Kansas_City: C_Kansas_City + O_Kansas_City = 24570

Demand_Los_Angeles: C_Los_Angeles + O_Los_Angeles = 64761

Demand_Glendale: C_Glendale + O_Glendale = 33689

Demand_Jacksonville: C_Jacksonville + O_Jacksonville = 684

In [ ]:
sum(O[i].varValue for i in range(len(O))) # Assuming O is a list-like object
# or if O has a defined length:
# sum(O[i].varValue for i in range(len(O)))

273522.0

## Question 2.b

In [ ]:
breakeven = (pulp.value(prob.objective))/(sum(C[i].varValue for i in range(len(C)))+sum(O[i].varValue for i in range(len(O))))
breakeven

1.7051679590237898

## Question 2.c

In [ ]:
prof_margin = breakeven*1.15
prof_margin

1.9609431528773582

## Question 2.d

In [ ]:
# Define original data
locations = [
    "Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
    "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"
]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Original costs
cincinnati_costs_ori = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
oakland_costs_ori = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]

cincinnati_production_cost = 1.20
oakland_production_cost = 1.65
cincinnati_capacity = 500000

# Function to adjust costs and solve
def solve_with_adjustment(adjustment):
    # Adjust freight costs
    adjustments = [-0.2, -0.1, 0, 0.1, 0.2]
    cincinnati_costs = [cost * (1 + adjustment) if cost < 1000 else cost for cost in cincinnati_costs_ori]
    oakland_costs = [cost * (1 + adjustment) if cost < 1000 else cost for cost in oakland_costs_ori]

    # Create LP problem
    prob = pulp.LpProblem("Question2_d", LpMinimize)

    # Decision variables
    cincinnati_vars = [pulp.LpVariable(f"Cincinnati_{loc}", lowBound=0) for loc in locations]
    oakland_vars = [pulp.LpVariable(f"Oakland_{loc}", lowBound=0) for loc in locations]

    # Objective function: Minimize total cost
    total_cost = (
        sum((cincinnati_costs[i] + cincinnati_production_cost) * cincinnati_vars[i] for i in range(len(locations)))
        + sum((oakland_costs[i] + oakland_production_cost) * oakland_vars[i] for i in range(len(locations)))
    )
    prob += total_cost

    # Constraints
    for i in range(len(locations)):
        prob += cincinnati_vars[i] + oakland_vars[i] == demand[i], f"Demand_{locations[i]}"
    prob += sum(cincinnati_vars) <= cincinnati_capacity, "Cincinnati_Capacity"

    # Solve the problem
    prob.solve()
    return pulp.value(prob.objective)

# Sensitivity analysis for -20%, -10%, 0%, +10%, +20% adjustments
adjustments = [-0.2, -0.1, 0, 0.1, 0.2]
results = {f"{int(adjustment * 100)}%": solve_with_adjustment(adjustment) for adjustment in adjustments}

# Print results
print("Freight Cost Sensitivity Analysis:\n")
for adjustment, total_cost in results.items():
    print(f"Change in Freight Costs: {adjustment} -> Total Cost: ${total_cost:,.2f}")

Freight Cost Sensitivity Analysis:

Change in Freight Costs: -20% -> Total Cost: $1,265,450.20
Change in Freight Costs: -10% -> Total Cost: $1,292,217.57
Change in Freight Costs: 0% -> Total Cost: $1,318,984.93
Change in Freight Costs: 10% -> Total Cost: $1,345,752.29
Change in Freight Costs: 20% -> Total Cost: $1,372,519.66


## Question 2.e

In [ ]:
# Define original data
locations = [
    "Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
    "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"
]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Original costs
cincinnati_costs_ori = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
oakland_costs_ori = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]

cincinnati_production_cost_ori = 1.20
oakland_production_cost_ori = 1.65
cincinnati_capacity = 500000

# Function to adjust costs and solve
def solve_with_cost_adjustments(prod_adj, ship_adj):
    # Adjust production and shipping costs
    cincinnati_production_cost = cincinnati_production_cost_ori * (1 + prod_adj)
    oakland_production_cost = oakland_production_cost_ori * (1 + prod_adj)
    cincinnati_costs = [cost * (1 + ship_adj) if cost < 1000 else cost for cost in cincinnati_costs_ori]
    oakland_costs = [cost * (1 + ship_adj) if cost < 1000 else cost for cost in oakland_costs_ori]

    # Create LP problem
    prob = pulp.LpProblem("Question2_e", LpMinimize)

    # Decision variables
    cincinnati_vars = [pulp.LpVariable(f"Cincinnati_{loc}", lowBound=0) for loc in locations]
    oakland_vars = [pulp.LpVariable(f"Oakland_{loc}", lowBound=0) for loc in locations]

    # Objective function: Minimize total cost
    total_cost = (
        sum((cincinnati_costs[i] + cincinnati_production_cost) * cincinnati_vars[i] for i in range(len(locations)))
        + sum((oakland_costs[i] + oakland_production_cost) * oakland_vars[i] for i in range(len(locations)))
    )
    prob += total_cost

    # Constraints
    for i in range(len(locations)):
        prob += cincinnati_vars[i] + oakland_vars[i] == demand[i], f"Demand_{locations[i]}"
    prob += sum(cincinnati_vars) <= cincinnati_capacity, "Cincinnati_Capacity"

    # Solve the problem
    prob.solve()
    return {
        "total_cost": pulp.value(prob.objective),
        "allocations": {
            locations[i]: {
                "Cincinnati": cincinnati_vars[i].varValue,
                "Oakland": oakland_vars[i].varValue
            }
            for i in range(len(locations))
        }
    }

# Sensitivity analysis for production and shipping cost adjustments
production_adjustments = [-0.1, 0, 0.1]  # -10%, 0%, +10% changes in production costs
shipping_adjustments = [-0.1, 0, 0.1]    # -10%, 0%, +10% changes in shipping costs

results = {}

for prod_adj in production_adjustments:
    for ship_adj in shipping_adjustments:
        scenario_name = f"Prod: {int(prod_adj * 100)}%, Ship: {int(ship_adj * 100)}%"
        results[scenario_name] = solve_with_cost_adjustments(prod_adj, ship_adj)

# Print results
print("Sensitivity Analysis for Production and Shipping Costs:\n")
for scenario, result in results.items():
    print(f"Scenario: {scenario}")
    print(f"  Total Cost: ${result['total_cost']:,.2f}")
    print("  Allocation:")
    for loc, alloc in result["allocations"].items():
        print(f"    {loc}: Cincinnati={alloc['Cincinnati']} gallons, Oakland={alloc['Oakland']} gallons")
    print("\n")


Sensitivity Analysis for Production and Shipping Costs:

Scenario: Prod: -10%, Ship: -10%
  Total Cost: $1,187,086.44
  Allocation:
    Santa Ana: Cincinnati=0.0 gallons, Oakland=22418.0 gallons
    El Paso: Cincinnati=6800.0 gallons, Oakland=0.0 gallons
    Pendleton: Cincinnati=39636.0 gallons, Oakland=40654.0 gallons
    Houston: Cincinnati=100447.0 gallons, Oakland=0.0 gallons
    Kansas City: Cincinnati=24570.0 gallons, Oakland=0.0 gallons
    Los Angeles: Cincinnati=0.0 gallons, Oakland=64761.0 gallons
    Glendale: Cincinnati=0.0 gallons, Oakland=33689.0 gallons
    Jacksonville: Cincinnati=68486.0 gallons, Oakland=0.0 gallons
    Little Rock: Cincinnati=148586.0 gallons, Oakland=0.0 gallons
    Bridgeport: Cincinnati=111475.0 gallons, Oakland=0.0 gallons
    Sacramento: Cincinnati=0.0 gallons, Oakland=112000.0 gallons


Scenario: Prod: -10%, Ship: 0%
  Total Cost: $1,213,853.80
  Allocation:
    Santa Ana: Cincinnati=0.0 gallons, Oakland=22418.0 gallons
    El Paso: Cincinnati=

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, value
import pandas as pd

# Define original data
locations = [
    "Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
    "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"
]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Original costs
cincinnati_costs_ori = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
oakland_costs_ori = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]

cincinnati_production_cost_ori = 1.20
oakland_production_cost_ori = 1.65
cincinnati_capacity = 500000

# Function to adjust costs and solve
def solve_with_cost_adjustments(prod_adj, ship_adj):
    # Adjust production and shipping costs
    cincinnati_production_cost = cincinnati_production_cost_ori * (1 + prod_adj)
    oakland_production_cost = oakland_production_cost_ori * (1 + prod_adj)
    cincinnati_costs = [cost * (1 + ship_adj) if cost < 1000 else cost for cost in cincinnati_costs_ori]
    oakland_costs = [cost * (1 + ship_adj) if cost < 1000 else cost for cost in oakland_costs_ori]

    # Create LP problem
    prob = LpProblem("Sensitivity_Analysis", LpMinimize)

    # Decision variables
    cincinnati_vars = [LpVariable(f"Cincinnati_{loc}", lowBound=0) for loc in locations]
    oakland_vars = [LpVariable(f"Oakland_{loc}", lowBound=0) for loc in locations]

    # Objective function: Minimize total cost
    total_cost = (
        sum((cincinnati_costs[i] + cincinnati_production_cost) * cincinnati_vars[i] for i in range(len(locations)))
        + sum((oakland_costs[i] + oakland_production_cost) * oakland_vars[i] for i in range(len(locations)))
    )
    prob += total_cost

    # Constraints
    for i in range(len(locations)):
        prob += cincinnati_vars[i] + oakland_vars[i] == demand[i], f"Demand_{locations[i]}"
    prob += sum(cincinnati_vars) <= cincinnati_capacity, "Cincinnati_Capacity"

    # Solve the problem
    prob.solve()
    return {
        "total_cost": value(prob.objective),
        "allocations": {
            locations[i]: {
                "Cincinnati": cincinnati_vars[i].varValue,
                "Oakland": oakland_vars[i].varValue
            }
            for i in range(len(locations))
        }
    }

# Sensitivity analysis for production and shipping cost adjustments
production_adjustments = [-0.2, -0.1, -0.05, 0, 0.05, 0.1, 0.2]  # -10%, -5%, 0%, 5%, 10% changes in production costs
shipping_adjustments = [-0.2, -0.1, -0.05, 0, 0.05, 0.1, 0.2]    # -10%, -5%, 0%, 5%, 10% changes in shipping costs

results = {}

for prod_adj in production_adjustments:
    for ship_adj in shipping_adjustments:
        scenario_name = f"Prod: {int(prod_adj * 100)}%, Ship: {int(ship_adj * 100)}%"
        results[scenario_name] = solve_with_cost_adjustments(prod_adj, ship_adj)

# Create a 5x5 matrix for production and shipping cost combinations
prod_levels = [-20,-10, -5, 0, 5, 10,20]  # Production adjustments (-10%, -5%, 0%, 5%, 10%)
ship_levels = [-20,-10, -5, 0, 5, 10,20]  # Shipping adjustments (-10%, -5%, 0%, 5%, 10%)

# Extract results into a 5x5 matrix
matrix_data = []
for prod in prod_levels:
    row = []
    for ship in ship_levels:
        scenario_name = f"Prod: {prod}%, Ship: {ship}%"
        row.append(results[scenario_name]['total_cost'])
    matrix_data.append(row)

# Convert to DataFrame for better presentation
matrix_df_5x5 = pd.DataFrame(
    matrix_data,
    index=[f"Prod: {prod}%" for prod in prod_levels],
    columns=[f"Ship: {ship}%" for ship in ship_levels]
)

# Ensure results are displayed with two decimal points
matrix_df_5x5 = pd.DataFrame(
    [[round(value, 2) for value in row] for row in matrix_data],
    index=[f"Prod: {prod}%" for prod in prod_levels],
    columns=[f"Ship: {ship}%" for ship in ship_levels]
)

# Display the 5x5 matrix in plain text format with 2 decimal points
print("Cost Sensitivity Matrix:\n")
print(matrix_df_5x5.to_string(index=True, header=True))


Cost Sensitivity Matrix:

            Ship: -20%  Ship: -10%   Ship: -5%    Ship: 0%    Ship: 5%   Ship: 10%   Ship: 20%
Prod: -20%  1055187.94  1081955.31  1095338.99  1108722.67  1122106.35  1134935.13  1160354.87
Prod: -10%  1160319.07  1187086.44  1200470.12  1213853.80  1227237.48  1240621.16  1267269.62
Prod: -5%   1212884.64  1239652.00  1253035.68  1266419.36  1279803.05  1293186.73  1319954.09
Prod: 0%    1265450.20  1292217.57  1305601.25  1318984.93  1332368.61  1345752.29  1372519.66
Prod: 5%    1318015.77  1344783.13  1358166.81  1371550.49  1384934.18  1398317.86  1425085.22
Prod: 10%   1370581.33  1397348.70  1410732.38  1424116.06  1437499.74  1450883.42  1477650.79
Prod: 20%   1475712.46  1502479.83  1515863.51  1529247.19  1542630.87  1556014.55  1582781.92


In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, value, LpStatus

# Define original data
locations = [
    "Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
    "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"
]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Original costs
cincinnati_costs = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
oakland_costs = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]

cincinnati_production_cost = 1.20
oakland_production_cost = 1.65
cincinnati_capacity = 500000

# Adjust the transportation cost for Cincinnati to El Paso by 20%
adjusted_cincinnati_costs = cincinnati_costs.copy()
adjusted_cincinnati_costs[locations.index("Pendleton")] *= 1.2

# Create LP problem
prob_adjusted = LpProblem("Adjusted_Cost", LpMinimize)

# Decision variables
C_adjusted = [LpVariable(f"C_{loc}", lowBound=0) for loc in locations]
O_adjusted = [LpVariable(f"O_{loc}", lowBound=0) for loc in locations]

# Objective function: Minimize total cost with adjusted cost
total_cost_adjusted = (
    sum((adjusted_cincinnati_costs[i] + cincinnati_production_cost) * C_adjusted[i] for i in range(len(locations)))
    + sum((oakland_costs[i] + oakland_production_cost) * O_adjusted[i] for i in range(len(locations)))
)
prob_adjusted += total_cost_adjusted

# Constraints
for i in range(len(locations)):
    prob_adjusted += C_adjusted[i] + O_adjusted[i] == demand[i], f"Demand_{locations[i]}"
prob_adjusted += sum(C_adjusted) <= cincinnati_capacity, "Cincinnati_Capacity"

# Solve the adjusted problem
prob_adjusted.solve()

# Display results
print("Status:", LpStatus[prob_adjusted.status])
print(f"The total cost after increasing by 20% \nin freight cost of the Cincinnati to Pendleton path: ${value(prob_adjusted.objective):,.2f}")
print("\nAllocation:")
for i in range(len(locations)):
    print(
        f"{locations[i]}: Cincinnati={C_adjusted[i].varValue} gallons, Oakland={O_adjusted[i].varValue} gallons"
    )


Status: Optimal
The total cost after increasing by 20% 
in freight cost of the Cincinnati to Pendleton path: $1,323,344.89

Allocation:
Santa Ana: Cincinnati=0.0 gallons, Oakland=22418.0 gallons
El Paso: Cincinnati=6800.0 gallons, Oakland=0.0 gallons
Pendleton: Cincinnati=0.0 gallons, Oakland=80290.0 gallons
Houston: Cincinnati=100447.0 gallons, Oakland=0.0 gallons
Kansas City: Cincinnati=24570.0 gallons, Oakland=0.0 gallons
Los Angeles: Cincinnati=0.0 gallons, Oakland=64761.0 gallons
Glendale: Cincinnati=0.0 gallons, Oakland=33689.0 gallons
Jacksonville: Cincinnati=68486.0 gallons, Oakland=0.0 gallons
Little Rock: Cincinnati=148586.0 gallons, Oakland=0.0 gallons
Bridgeport: Cincinnati=111475.0 gallons, Oakland=0.0 gallons
Sacramento: Cincinnati=0.0 gallons, Oakland=112000.0 gallons


In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, value, LpStatus

# Define original data
locations = [
    "Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
    "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"
]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Original costs
cincinnati_costs = [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000]
#oakland_costs = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]
oakland_costs = [0.22*0.9, 0.74*0.9, 0.49*0.9, 1000, 1000, 0.22*0.9, 0.22*0.9, 1000, 1000, 1000, 0.15*0.9]

oakland_production_cost = 1.65
cincinnati_production_cost = 1.20*1.1
cincinnati_capacity = 500000

# Adjust the production cost for Oakland to El Paso by 10%
adjusted_oakland_production_costs = oakland_costs.copy()
adjusted_oakland_production_costs[locations.index("El Paso")] = oakland_production_cost

# Create LP problem
prob_adjusted = LpProblem("Adjusted_Cost", LpMinimize)

# Decision variables
C_adjusted = [LpVariable(f"C_{loc}", lowBound=0) for loc in locations]
O_adjusted = [LpVariable(f"O_{loc}", lowBound=0) for loc in locations]

# Objective function: Minimize total cost with adjusted cost
total_cost_adjusted = (
    sum((cincinnati_costs[i] + cincinnati_production_cost) * C_adjusted[i] for i in range(len(locations)))
    + sum((oakland_costs[i] + (adjusted_oakland_production_costs[i] if i == locations.index("El Paso") else oakland_production_cost)) * O_adjusted[i] for i in range(len(locations)))
)
prob_adjusted += total_cost_adjusted

# Constraints
for i in range(len(locations)):
    prob_adjusted += C_adjusted[i] + O_adjusted[i] == demand[i], f"Demand_{locations[i]}"
prob_adjusted += sum(C_adjusted) <= cincinnati_capacity, "Cincinnati_Capacity"

# Solve the adjusted problem
prob_adjusted.solve()

# Display results
print("Status:", LpStatus[prob_adjusted.status])
print(f"The total cost after increasing by 10% in production cost of the Cincinnati\nand decreasing by 10% in freight costs of Oakland: ${value(prob_adjusted.objective):,.2f}")
print("\nAllocation:")
for i in range(len(locations)):
    print(
        f"{locations[i]}: Cincinnati={C_adjusted[i].varValue} gallons, Oakland={O_adjusted[i].varValue} gallons"
    )


Status: Optimal
The total cost after increasing by 10% in production cost of the Cincinnati
and decreasing by 10% in freight costs of Oakland: $1,370,315.26

Allocation:
Santa Ana: Cincinnati=0.0 gallons, Oakland=22418.0 gallons
El Paso: Cincinnati=6800.0 gallons, Oakland=0.0 gallons
Pendleton: Cincinnati=0.0 gallons, Oakland=80290.0 gallons
Houston: Cincinnati=100447.0 gallons, Oakland=0.0 gallons
Kansas City: Cincinnati=24570.0 gallons, Oakland=0.0 gallons
Los Angeles: Cincinnati=0.0 gallons, Oakland=64761.0 gallons
Glendale: Cincinnati=0.0 gallons, Oakland=33689.0 gallons
Jacksonville: Cincinnati=68486.0 gallons, Oakland=0.0 gallons
Little Rock: Cincinnati=148586.0 gallons, Oakland=0.0 gallons
Bridgeport: Cincinnati=111475.0 gallons, Oakland=0.0 gallons
Sacramento: Cincinnati=0.0 gallons, Oakland=112000.0 gallons


In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, value, LpStatus

# Define original data
locations = [
    "Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
    "Los Angeles", "Glendale", "Jacksonville", "Little Rock", "Bridgeport", "Sacramento"
]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Original costs
cincinnati_costs = [1000, 0.84*1.1, 0.83*1.1, 0.45*1.1, 0.36*1.1, 1000, 1000, 0.34*1.1, 0.34*1.1, 0.34*1.1, 1000]
#oakland_costs = [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]
oakland_costs = [0.22*0.9, 0.74*0.9, 0.49*0.9, 1000, 1000, 0.22*0.9, 0.22*0.9, 1000, 1000, 1000, 0.15*0.9]

oakland_production_cost = 1.65*0.9
cincinnati_production_cost = 1.20*1.1
cincinnati_capacity = 500000

# Adjust the production cost for Oakland to El Paso by 10%
adjusted_oakland_production_costs = oakland_costs.copy()
adjusted_oakland_production_costs[locations.index("El Paso")] = oakland_production_cost

# Create LP problem
prob_adjusted = LpProblem("Adjusted_Cost", LpMinimize)

# Decision variables
C_adjusted = [LpVariable(f"C_{loc}", lowBound=0) for loc in locations]
O_adjusted = [LpVariable(f"O_{loc}", lowBound=0) for loc in locations]

# Objective function: Minimize total cost with adjusted cost
total_cost_adjusted = (
    sum((cincinnati_costs[i] + cincinnati_production_cost) * C_adjusted[i] for i in range(len(locations)))
    + sum((oakland_costs[i] + (adjusted_oakland_production_costs[i] if i == locations.index("El Paso") else oakland_production_cost)) * O_adjusted[i] for i in range(len(locations)))
)
prob_adjusted += total_cost_adjusted

# Constraints
for i in range(len(locations)):
    prob_adjusted += C_adjusted[i] + O_adjusted[i] == demand[i], f"Demand_{locations[i]}"
prob_adjusted += sum(C_adjusted) <= cincinnati_capacity, "Cincinnati_Capacity"

# Solve the adjusted problem
prob_adjusted.solve()

# Display results
print("Status:", LpStatus[prob_adjusted.status])
print(f"The total cost after increasing by 10% in production cost and freight costs of the Cincinnati\nand decreasing by 10% in production cost and freight costs of Oakland: ${value(prob_adjusted.objective):,.2f}")
print("\nAllocation:")
for i in range(len(locations)):
    print(
        f"{locations[i]}: Cincinnati={C_adjusted[i].varValue} gallons, Oakland={O_adjusted[i].varValue} gallons"
    )


Status: Optimal
The total cost after increasing by 10% in production cost and freight costs of the Cincinnati
and decreasing by 10% in production cost and freight costs of Oakland: $1,335,158.23

Allocation:
Santa Ana: Cincinnati=0.0 gallons, Oakland=22418.0 gallons
El Paso: Cincinnati=0.0 gallons, Oakland=6800.0 gallons
Pendleton: Cincinnati=0.0 gallons, Oakland=80290.0 gallons
Houston: Cincinnati=100447.0 gallons, Oakland=0.0 gallons
Kansas City: Cincinnati=24570.0 gallons, Oakland=0.0 gallons
Los Angeles: Cincinnati=0.0 gallons, Oakland=64761.0 gallons
Glendale: Cincinnati=0.0 gallons, Oakland=33689.0 gallons
Jacksonville: Cincinnati=68486.0 gallons, Oakland=0.0 gallons
Little Rock: Cincinnati=148586.0 gallons, Oakland=0.0 gallons
Bridgeport: Cincinnati=111475.0 gallons, Oakland=0.0 gallons
Sacramento: Cincinnati=0.0 gallons, Oakland=112000.0 gallons
